In [2]:
import cv2 # OpenCV library
import numpy as np # Numpy library for scientific computing
import matplotlib.pyplot as plt # Matplotlib library for plotting

#Default dimensions taken by overhead camera
IMAGE_WIDTH = 3840
IMAGE_HEIGHT = 2160

#Corners of the maze in this image determined by image viewing software. This is the same for all images as the location of the camera relative to the maze is always the same
TOP_LEFT_CORNER = np.float32([450,300])
TOP_RIGHT_CORNER = np.float32([3400,300])
BOTTOM_LEFT_CORNER = np.float32([450,1900])
BOTTOM_RIGHT_CORNER = np.float32([3400,1900])

# each section is 426 x 432 pixels 
SECTION_WIDTH = 426
SECTION_HEIGHT = 432
WALL_RATIO = 0.05


h_start_pixel = 0
h_end_pixel = SECTION_WIDTH
v_start_pixel = 0
v_end_pixel = SECTION_HEIGHT
h_section_pixel = SECTION_WIDTH
v_section_pixel = SECTION_HEIGHT


def determine_wall_ratio(start1, end1, start2, end2, pic):
    wall_count = 0
    no_wall_count = 0
    for v in range(start1,end1):
        for h in range(start2,end2):
            px_hsv_value = pic[h,v]
            if (px_hsv_value > 220):
                wall_count+=1
            else:
                no_wall_count+=1
    return wall_count / no_wall_count


 

# Blank maze is initialised as a list as Python strings are immutable
maze_list = [[' ','-','-','-',' ','-','-','-',' ','-','-','-',' ','-','-','-',' ','-','-','-',' ','-','-','-',' ','-','-','-',' ','-','-','-',' ','-','-','-',' '],
             ['|',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','|'],
             [' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' '],
             ['|',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','|'],
             [' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' '],
             ['|',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','|'],
             [' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' '],
             ['|',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','|'],
             [' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' '],
             ['|',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','|'],
             [' ','-','-','-',' ','-','-','-',' ','-','-','-',' ','-','-','-',' ','-','-','-',' ','-','-','-',' ','-','-','-',' ','-','-','-',' ','-','-','-',' ']]




# Taking in raw image from camera and cropping it such that only the maze itself is visible
img_bgr = cv2.imread('webcam_8.jpg')
pts1 = np.float32([TOP_LEFT_CORNER,TOP_RIGHT_CORNER,BOTTOM_LEFT_CORNER,BOTTOM_RIGHT_CORNER]) 
pts2 = np.float32([[0,0],[IMAGE_WIDTH,0],[0,IMAGE_HEIGHT],[IMAGE_WIDTH,IMAGE_HEIGHT]]) 
H = cv2.getPerspectiveTransform(pts1,pts2)
dst = cv2.warpPerspective(img_bgr, H, (IMAGE_WIDTH,IMAGE_HEIGHT))
hsv = cv2.cvtColor(dst, cv2.COLOR_BGR2HSV)
lower_blue = np.array([0,0,0])
upper_blue = np.array([180,255,100])
final = cv2.inRange(hsv, lower_blue, upper_blue)




# Analysing vertical walls
for row in range(0,5):
    for col in range(0,8):
        # Creating sections where a wall can be placed and iterate through the pixels in this section
        h_section_pixel = SECTION_WIDTH*col + SECTION_WIDTH + col*9 # Corrective factor included to account for distortion, misalignment and rounding
        h_start_section = h_section_pixel - 20
        h_end_section = h_section_pixel + 20
        vertical_walls_pixel_ratio = determine_wall_ratio(h_start_section,h_end_section,v_start_pixel,v_end_pixel,final)
        # If the ratio of wall coloured pixels to non-wall coloured pixels are greater than our defined constant, there is a wall here
        if (vertical_walls_pixel_ratio > WALL_RATIO):            
            maze_list[2*row+1][4*col+4] = '|'
            image = cv2.rectangle(dst, (h_start_section,v_start_pixel), (h_end_section,v_end_pixel), (255,0,0), 5) # debugging

    v_start_pixel = v_end_pixel
    v_end_pixel+=SECTION_HEIGHT


# Analysing horizontal walls
for col in range(0,9):
    for row in range(0,4):
        # Creating sections where a wall can be placed and iterate through the pixels in this section
        v_section_pixel = (SECTION_HEIGHT)*row + SECTION_HEIGHT + 15*(row) # Corrective factor included to account for distortion, misalignment and rounding
        v_start_section = v_section_pixel - 20
        v_end_section = v_section_pixel + 20
        horizontal_walls_pixel_ratio = determine_wall_ratio(h_start_pixel, h_end_pixel, v_start_section,v_end_section, final)

        # If the ratio of wall coloured pixels to non-wall coloured pixels are greater than our defined constant, there is a wall here
        if (row == 0):
            print(horizontal_walls_pixel_ratio)
        if (horizontal_walls_pixel_ratio > WALL_RATIO):
            maze_list[2*row+2][4*col+1] = '-'
            maze_list[2*row+2][4*col+2] = '-'
            maze_list[2*row+2][4*col+3] = '-'
            image = cv2.rectangle(dst, (h_start_pixel,v_start_section), (h_end_pixel, v_end_section), (255,0,0), 3) #debugging

    h_start_pixel = h_end_pixel
    h_end_pixel+=SECTION_WIDTH



for x in maze_list:
    print("".join(x))


plt.imshow(dst) # for debugging
plt.show() # for debugging purposes


ZeroDivisionError: division by zero